### MLP 모델

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(MultilayerPerceptron,self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x_in, apply_softmax=False):
        
#         relu 활성화 함수를 통해 비선형성을 추가
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        
#         softmax 적용여부 
        if apply_softmax:
#           dim = 행렬의 각 행에 대해 소프트맥스 함수를 적용
            output = F.softmax(output, dim=1)
        return output

In [2]:
# example1

input_dim = 3
hidden_dim = 100
output_dim = 4

mlp = MultilayerPerceptron(input_dim,hidden_dim,output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


# 예제: MLP로 성씨 분류하기

### 데이터 로드

In [3]:
import pandas as pd

df = pd.read_csv("surnames_with_splits.csv")
df


,nationality,nationality_index,split,surname
0,Arabic,15,train,Totah
1,Arabic,15,train,Abboud
2,Arabic,15,train,Fakhoury
3,Arabic,15,train,Srour
4,Arabic,15,train,Sayegh
...,...,...,...,...
10975,Vietnamese,11,test,Dinh
10976,Vietnamese,11,test,Phung
10977,Vietnamese,11,test,Quang
10978,Vietnamese,11,test,Vu


In [4]:
df['nationality'].value_counts()

nationality
English       2972
Russian       2373
Arabic        1603
Japanese       775
Italian        600
German         576
Czech          414
Spanish        258
Dutch          236
French         229
Chinese        220
Irish          183
Greek          156
Polish         120
Korean          77
Scottish        75
Vietnamese      58
Portuguese      55
Name: count, dtype: int64

In [5]:
df['split'].value_counts()

split
train    7680
test     1660
val      1640
Name: count, dtype: int64

### 데이터 split(train/valid/test)

In [6]:
# 데이터를 다시 train/valid/test로 나눠줌

# train 데이터 
train_df = df[df.split=='train']
train_size = len(train_df)

# valid 데이터 
val_df = df[df.split=='val']
val_size = len(val_df)

# test 데이터 
test_df = df[df.split=='test']
test_size = len(test_df)

### 2. Vocabulary

In [7]:
from collections import Counter
import string

# Counter()를 통해 어떤 단어가 얼만큼의 횟수로 들어있는지를 알 수 있다.
word_counts = Counter()
for name_text in df.surname:
    for word in name_text.split(" "):
        # word가 .(구두점,punctuation)이 아닐 경우 word에 추가
        if word not in string.punctuation:
            word_counts[word] += 1

# word_counts

In [8]:
# add_unk=True를 하면 '<UNK>': 0 토큰을 추가해줌 !

class Vocabulary:
    def __init__(self, add_unk=False):
        self.token_to_idx = {}
        self.idx_to_token = {}
    
#         "UNK" 토큰이 추가되지 않는 경우에는 -1로 설정,
        self.unk_index = -1
        if add_unk:
#         "UNK" 토큰이 추가될 경우에는 UNK에 해당하는 인덱스로 설정,
            self.unk_index = self.add_token('<UNK>') 

    def add_token(self, token):
        
#       만약 해당 토큰이 있으면 토큰 idx만 return
        if token in self.token_to_idx:
            index = self.token_to_idx[token]
            
#       만약 해당 토큰이 없으면 새로운 토큰 만들어줌
        else:
            index = len(self.token_to_idx)
            self.token_to_idx[token] = index
            self.idx_to_token[index] = token
        return index

In [9]:
# cutoff 보다 수가 많은 단어만 vocab에 추가
cutoff = 0

# Vocabulary 객체 생성
# cutoff보다 작으면 unk토큰으로 지정해줄 것이기 때문에 True
name_vocab = Vocabulary(add_unk=True)

# word_counts.items() -> ex) ('all', 24160)
for word, count in word_counts.items():
    if count > cutoff:
        name_vocab.add_token(word)

In [10]:
print(dict(list(name_vocab.token_to_idx.items())[:5]))

{'<UNK>': 0, 'Totah': 1, 'Abboud': 2, 'Fakhoury': 3, 'Srour': 4}


In [11]:
print(dict(list(name_vocab.idx_to_token.items())[:5]))

{0: '<UNK>', 1: 'Totah', 2: 'Abboud', 3: 'Fakhoury', 4: 'Srour'}


### 국적 Vocabulary

In [12]:
nation_vocab = Vocabulary(add_unk=False)
nation_vocab

In [13]:
# 국적 Vocabulary 추가 

for nation in sorted(set(df.nationality)):
    nation_vocab.add_token(nation)
#     print(nation)

In [14]:
nation_vocab.token_to_idx

{'Arabic': 0,
 'Chinese': 1,
 'Czech': 2,
 'Dutch': 3,
 'English': 4,
 'French': 5,
 'German': 6,
 'Greek': 7,
 'Irish': 8,
 'Italian': 9,
 'Japanese': 10,
 'Korean': 11,
 'Polish': 12,
 'Portuguese': 13,
 'Russian': 14,
 'Scottish': 15,
 'Spanish': 16,
 'Vietnamese': 17}

In [15]:
print(dict(list(nation_vocab.idx_to_token.items())[:5]))

{0: 'Arabic', 1: 'Chinese', 2: 'Czech', 3: 'Dutch', 4: 'English'}


## 3. Vectorizer

In [16]:
# 주어진 토큰에 대응하는 인덱스 반환

def lookup_token(vocabulary_class,token):

# UNK 토큰이 있을 경우
    if vocabulary_class.unk_index >= 0:
#           토큰을 찾아보고 없으면 unk_index 반환, 있으면 해당 토큰의 idx를 반환
        return vocabulary_class.token_to_idx.get(token, vocabulary_class.unk_index)
    else:
        return vocabulary_class.token_to_idx[token]
    

In [17]:
# 주어진 인덱스에 대응하는 토큰 반환

def lookup_index(vocabulary_class, index):
        if index not in vocabulary_class.idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return vocabulary_class.idx_to_token[index]
    

### 텍스트(surname)에 대한 원 핫 인코딩

In [18]:
import numpy as np 

def vectorize(voca, column):

#     전체 단어 사이즈만큼을 미리 0으로 채워둠
    one_hot = np.zeros(len(voca.token_to_idx), dtype=np.float32)
    one_hot
    
    for token in column.split(" "):
        
#         토큰이 .(구두점)이 아닐 경우 
#         토큰에 해당되는 인덱스에 1를 부여한 one_hot encoding 만듦
        if token not in string.punctuation:
            one_hot[lookup_token(voca,token)] = 1

    return one_hot

print(vectorize(name_vocab,"all i can say is that a i had no other option"))

[1. 0. 0. ... 0. 0. 0.]


### Dataset class

In [19]:
import torch
from torch.utils.data import Dataset

class NameDataset(Dataset):
    def __init__(self, names, nations):
        self.names = names
        self.nations = nations

    def __len__(self):
        return len(self.names)

    def __getitem__(self, index):
        name = self.names[index]
        nation = self.nations[index]
        
#         여기다가 vectorize함수 사용해서 name return
        vectorized_name = vectorize(name_vocab,name)
#       nation 숫자로 return
#         vectorized_nation = vectorize(nation_vocab,nation)
        vectorized_nation = lookup_token(nation_vocab,nation)
    
        return {
            'surname': vectorized_name,
            'nationality': vectorized_nation
        }

### 데이터셋 class

In [20]:
# 데이터셋을 인스턴스화 해주어야 로더에 넣어줄 수 있다. 

train_dataset = NameDataset(train_df["surname"].values, train_df["nationality"].values)
train_dataset

valid_dataset = NameDataset(val_df["surname"].values, val_df["nationality"].values)
valid_dataset

test_dataset = NameDataset(test_df["surname"].values, test_df["nationality"].values)
test_dataset


In [21]:
# 데이터 로더 설정
from torch.utils.data import DataLoader

# drop_last=True -> 배치 사이즈보다 over하면 drop

Traindataloader = DataLoader(dataset=train_dataset, batch_size=512,
                            shuffle=True, drop_last=True)

Validdataloader = DataLoader(dataset=valid_dataset, batch_size=512,
                            shuffle=False, drop_last=True)

Testdataloader = DataLoader(dataset=test_dataset, batch_size=512,
                            shuffle=False, drop_last=True)


In [22]:
print(len(train_dataset),len(Traindataloader))

7680 15


In [23]:
for batch_index, batch_dict in enumerate(Traindataloader):
    print(batch_index)
    print(batch_dict)
    
    break
    

0
{'surname': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'nationality': tensor([ 0, 14,  2,  1, 14,  4,  4,  7,  9, 16,  4,  2,  2,  5, 14, 12,  6,  0,
         4, 10,  2,  4,  6, 14, 14,  0,  2, 10,  0,  4,  4,  8,  0,  4,  4,  0,
         4,  4,  5, 14, 10,  4,  4,  6, 14,  6,  6,  4,  0,  4,  6,  9, 10,  4,
         4,  0,  3,  4,  4, 14,  4, 14,  0,  9,  4, 14, 10,  4,  4, 10, 14,  4,
         4,  9, 10,  8, 14, 14,  0, 16,  4,  4,  4, 14,  0,  0,  1, 14, 14,  2,
        14, 14,  4,  0,  0,  0, 14,  0, 14, 14,  4,  0,  5, 10, 14, 10,  0,  8,
         0,  0,  4,  4,  4,  4, 14,  0,  0, 10, 14, 14, 14,  4, 14,  0, 14,  4,
         4,  0,  4,  5,  4,  4,  4, 10, 14, 10, 14,  2,  6, 10, 16, 14,  4, 16,
         6,  5,  0,  9, 14, 14,  2,  4,  2,  4,  4,  4,  1,  4,  0, 14, 11, 

### 모델 정의 및 옵티마이저, loss func 설정

### 모델정의 NameClassifier

In [24]:
# nn은 neural network로 torch의 신경망 모듈이다.
# MLP사용 
import torch.nn as nn
import torch.nn.functional as F

class NameClassifier(nn.Module):
    
    def __init__(self,input_dim, hidden_dim, output_dim):
        
#       torch.nn.Module의 초기화 메서드를 실행하여 해당 클래스의 기능을 상속받음
        super(NameClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim,hidden_dim)
        self.fc2 = nn.Linear(hidden_dim,output_dim)
    
    def forward(self, x_in, apply_softmax=False):
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        
        if apply_softmax:
            output = F.softmax(output,dim=1)
        
        return output
        

### 드롭 아웃 적용한 모델

In [25]:
# nn은 neural network로 torch의 신경망 모듈이다.
import torch.nn as nn
import torch.nn.functional as F

class NameClassifier(nn.Module):
    
    def __init__(self,input_dim, hidden_dim, output_dim):
        
#       torch.nn.Module의 초기화 메서드를 실행하여 해당 클래스의 기능을 상속받음
        super(NameClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim,hidden_dim)
        self.fc2 = nn.Linear(hidden_dim,output_dim)
    
    def forward(self, x_in, apply_softmax=False):
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate,p=0.5))
        
        if apply_softmax:
            output = F.softmax(output,dim=1)
        
        return output
        

In [26]:
len(name_vocab.token_to_idx)

9042

In [27]:
len(nation_vocab.token_to_idx)

18

In [28]:
hidden_dim = 300

classifier = NameClassifier(input_dim=len(name_vocab.token_to_idx),
                            hidden_dim=hidden_dim,
                           output_dim=len(nation_vocab.token_to_idx))
classifier

NameClassifier(
  (fc1): Linear(in_features=9042, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=18, bias=True)
)

### 옵티마이저, loss function

In [29]:
lr = 0.001
num_epochs = 100

In [30]:
# 옵티마이저
import torch.optim as optim

optimizer = optim.Adam(classifier.parameters(), lr = lr)
optimizer


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [31]:
df['nationality'].value_counts()

nationality
English       2972
Russian       2373
Arabic        1603
Japanese       775
Italian        600
German         576
Czech          414
Spanish        258
Dutch          236
French         229
Chinese        220
Irish          183
Greek          156
Polish         120
Korean          77
Scottish        75
Vietnamese      58
Portuguese      55
Name: count, dtype: int64

In [32]:
numSample_list = df['nationality'].value_counts().tolist()
numSample_list
# weights 계산
weights = [1 - (x / sum(numSample_list)) for x in numSample_list]

# weights를 torch.FloatTensor로 변환
weights = torch.FloatTensor(weights)
weights

tensor([0.7293, 0.7839, 0.8540, 0.9294, 0.9454, 0.9475, 0.9623, 0.9765, 0.9785,
        0.9791, 0.9800, 0.9833, 0.9858, 0.9891, 0.9930, 0.9932, 0.9947, 0.9950])

In [33]:
# loss function
#  dataset.class_weights -> 각 클래스에 대해 다른 가중치를 적용할 수 있음(데이터 불균형시에)
# 소수의 클래스가 다수의 클래스보다 훨씬 적은 수의 샘플을 가지고 있는 경우, 
# 소수 클래스에 더 높은 가중치를 부여하여 모델이 불균형한 데이터에 대해 더 잘 학습

loss_func = nn.CrossEntropyLoss(weights)
loss_func


CrossEntropyLoss()

### Train

In [34]:
def compute_accuracy(y_pred, y_target):
#      예측값과 타겟값을 비교하여 일치하는 개수를 계산
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100
   

In [35]:
# Train state 초기화 
def make_train_state():
    return {
        'stop_early':False,
        'early_stopping_step':0,
        'early_stopping_best_val':1e8,
        'early_stopping_criteria' : 10,
        'epoch_index' : 0,
        'train_loss': [], 
        'train_acc' :[], 
        'val_loss' : [],
        'val_acc' : [], 
        'test_loss' : [],
        'test_acc' : [],
         
#       모델 저장파일
        'model_filename' : 'model.pth'
    } 


# Train update 
def update_train_state(model, train_state):
    
#   학습시작하면 초기에 모델 저장하기 
    
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(),train_state['model_filename'])
        
#   모델 성능이 향상되면 모델 저장(valid loss가 더 낮아지면)
    elif train_state['epoch_index'] >=1 :
        loss_t = train_state['val_loss'][-1]
#        loss가 나빠지면 early stop step 업데이트
        if loss_t >= train_state['early_stopping_best_val']:
            train_state['early_stopping_step']+=1
            
#        loss가 좋아지면   
        else:
#            early stop step 0으로 다시 초기화        
            train_state['early_stopping_step']=0
    
#           최저 loss이면 모델 저장 
            if loss_t < train_state['early_stopping_best_val']:
                train_state['early_stopping_best_val'] = loss_t
                torch.save(model.state_dict(),train_state['model_filename'])

#       기준점 넘으면 early stop 
        if train_state['early_stopping_step'] >= train_state['early_stopping_criteria']:
            train_state['stop_early'] = True
        
        return train_state


In [36]:
# 모델 진행 상황 함수 초기화
train_state = make_train_state()
train_state

{'stop_early': False,
 'early_stopping_step': 0,
 'early_stopping_best_val': 100000000.0,
 'early_stopping_criteria': 10,
 'epoch_index': 0,
 'train_loss': [],
 'train_acc': [],
 'val_loss': [],
 'val_acc': [],
 'test_loss': [],
 'test_acc': [],
 'model_filename': 'model.pth'}

In [37]:
import tqdm

# 에포크만큼
for epoch in tqdm.tqdm(range(num_epochs)):

#     print('epoch',epoch)
#     print(train_state['epoch_index']) 
    train_state['epoch_index'] +=1 

    running_loss = 0.0
    running_acc = 0.0


#     모델을 학습 모드로 설정 -> 드롭아웃 및 배치 정규화와 같은 학습 중에만 적용되는 기법들이 활성화
#     모델을 평가 모드로 전환하려면 classifier.eval()을 사용
    classifier.train()
# 배치 만큼
    for batch_idx, batch_data in enumerate(Traindataloader):

        

#       1. 옵티마이저 그레디언트 0으로 초기화
        optimizer.zero_grad()
#       2. 모델에 데이터 넣어서 출력받기
        y_pred = classifier(x_in=batch_data['surname'])
#       3. loss 계산하기
        loss =  loss_func(y_pred, batch_data['nationality'])
    
#       tensor(0.3190) -> 0.3190, item()으로 스칼라 값만 추출
        loss_t = loss.item()

#       배치에서의 평균 loss 구하기
        running_loss += (loss_t - running_loss) / (batch_idx + 1)

#       4. gradient 계산하기
        loss.backward()

#       5. 옵티마이저 가중치 업데이트
        optimizer.step()

#       Accuracy 계산
        acc_t = compute_accuracy(y_pred, batch_data['nationality'])
        running_acc += (acc_t - running_acc) / (batch_idx + 1)



    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)


#   valid에 대한 계산

    running_loss = 0.0
    running_acc = 0.0

    classifier.eval() # 모델 파라미터를 수정하지 못 하게 비활성화

    for batch_idx, batch_data in enumerate(Validdataloader):

#       1. 모델의 출력값(y_pred)계산
        y_pred = classifier(x_in=batch_data['surname'])

#       2. loss 계산
        loss = loss_func(y_pred,batch_data['nationality'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_idx + 1)

#       3. Accuracy 계산
        acc_t = compute_accuracy(y_pred,batch_data['nationality'])
        running_acc += (acc_t - running_acc) / (batch_idx + 1)
    
    print("val_loss",running_loss)
    print("val_acc",running_acc)

    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)
    

#   전체 loss, acc 저장
    train_state = update_train_state(model=classifier,
                                     train_state=train_state)
#   early stop해라고 했으면 학습 멈추기    
    if train_state['stop_early']:
        break



  1%|▍                                          | 1/100 [00:01<01:47,  1.08s/it]

val_loss 2.8303771018981934
val_acc 37.6953125


  2%|▊                                          | 2/100 [00:01<01:35,  1.03it/s]

val_loss 2.6598594983418784
val_acc 40.75520833333333


  3%|█▎                                         | 3/100 [00:02<01:31,  1.06it/s]

...

val_loss 1.847023884455363
val_acc 43.098958333333336


 23%|█████████▋                                | 23/100 [00:21<01:03,  1.21it/s]

val_loss 1.8597977956136067
val_acc 42.447916666666664


 23%|█████████▋                                | 23/100 [00:22<01:13,  1.04it/s]

val_loss 1.8799113829930623
val_acc 40.8203125


### Test 진행

In [38]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산합니다

classifier.load_state_dict(torch.load(train_state['model_filename']))

running_loss = 0.0
running_acc = 0.0

# 가중치 업데이트 하지 못 하게
classifier.eval()

for batch_idx, batch_data in enumerate(Testdataloader):
    
    y_pred = classifier(x_in=batch_data['surname'])
    loss = loss_func(y_pred,batch_data['nationality'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_idx + 1)
    
    acc_t = compute_accuracy(y_pred, batch_data['nationality'])
    running_acc += (acc_t - running_acc) / (batch_idx + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [39]:
print("테스트 손실: {:.3f}".format(train_state['test_loss']))
print("테스트 정확도: {:.2f}".format(train_state['test_acc']))

테스트 손실: 1.849
테스트 정확도: 42.06


In [40]:
train_state

{'stop_early': True,
 'early_stopping_step': 10,
 'early_stopping_best_val': 1.810646692911784,
 'early_stopping_criteria': 10,
 'epoch_index': 24,
 'train_loss': [2.8753815809885666,
  2.758050107955932,
  2.5297334353129064,
  2.2607335567474367,
  2.0937030315399165,
  1.9954595883687338,
  1.888177291552226,
  1.7716776053110759,
  1.6456313212712605,
  1.5146377881368,
  1.3768490552902222,
  1.2391159057617185,
  1.100442369778951,
  0.9662196358044941,
  0.847908357779185,
  0.7403143843015035,
  0.6467431187629701,
  0.5703393757343291,
  0.5004418730735779,
  0.4429114580154419,
  0.39431544542312624,
  0.3511805991331736,
  0.31422823468844097,
  0.2831361959377925],
 'train_acc': [15.520833333333334,
  39.609375,
  41.95312499999999,
  37.552083333333336,
  32.70833333333333,
  33.333333333333336,
  46.510416666666664,
  51.236979166666664,
  58.91927083333333,
  62.09635416666667,
  63.65885416666667,
  69.140625,
  77.89062499999999,
  82.17447916666667,
  84.8828125,
  87

### 추론

In [41]:
new_surname = "Kim"

In [42]:
# 벡터화 + 텐서화

vectorized_surname = torch.tensor(vectorize(name_vocab,new_surname))
vectorized_surname

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [43]:
print(vectorized_surname.shape)

torch.Size([9042])


In [44]:
# 첫번째 차원을 1로 만들고, 나머지는 다른 차원으로 알아서 되도록 !
x_data = vectorized_surname.view(1, -1)
print(x_data.shape)

torch.Size([1, 9042])


In [45]:
# 모델에 test 데이터 넣어주기
result = classifier(x_data,apply_softmax=True)
print(result)

tensor([[0.0216, 0.0386, 0.0752, 0.0630, 0.0868, 0.0655, 0.1066, 0.0363, 0.0484,
         0.0977, 0.1037, 0.0252, 0.0261, 0.0165, 0.0873, 0.0238, 0.0595, 0.0183]],
       grad_fn=<SoftmaxBackward0>)


In [46]:
result.max()

tensor(0.1066, grad_fn=<MaxBackward1>)

In [47]:
probability_value, indices = result.max(dim=1)

print("probability_value",probability_value)
print("indices",indices)

probability_value tensor([0.1066], grad_fn=<MaxBackward0>)
indices tensor([6])


In [48]:
print("새로운 데이터에 대해 추론한 label",indices.item())

새로운 데이터에 대해 추론한 label 6


In [49]:
prediction = lookup_index(nation_vocab,indices.item())
prediction

'German'

In [50]:
print("{} -> {}".format(new_surname, prediction))

Kim -> German
